In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import log
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import LineCollection

print("Current working directory:", os.getcwd())

Current working directory: c:\Users\VF6603\OneDrive - ENGIE\Escritorio\Codes\TFG


# Análisis de LCEs vs $\sigma$
Hacemos barridos para distintos de valores de $x_0$

In [ ]:
# definimos los parametros del sistema
r = np.array(
    [1, 0.72, 1.53, 1.27]
)

a = np.array([
    [1, 1.09, 1.52, 0],
    [0, 1, 0.44, 1.36],
    [2.33, 0, 1, 0.47],
    [1.21, 0.51, 0.35, 1]
])

x0 = np.array([0.5, 0.5, 0.5, 0.5])

# generamos un nuevo x0:
rng = np.random.default_rng(73)
x0 = np.array(
    [[rng.random() for _ in range(4)]]
)
print

array([[0.49263619, 0.30336754, 0.26856778, 0.98515715]])

In [20]:
(5000/1e5)

0.05

In [21]:
# PRUEBA DE TIEMPO DE UN WOLF
from src.stoc_system import StochasticSystem
model = StochasticSystem(a, r, x0, 5000, 5000/1e5, [0.04, 0.04, 0.04, 0.04])
t, x = model.euler_maruyama()
lces = model.estimate_LCE1_Wolf(
            x, evol_time=500,
            min_separation=180,
            max_replacements=500,
            start_indices= 1e5 // 2
        )
print(lces[0])

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def barrido_LCEs(total_time = 5000, N_points=1e5, max_iters=50, save=False, save_path=None, save_name=None):
    """Función que realiza un barrido de diferentes intensidades de ruido del sistema y puede guadar los resultados.
    :param total_time: Tiempo total de simulación
    :param N_points: Número de puntos para el método de Euler–Maruyama
    :param max_iters: Número máximo de iteraciones para el barrido
    :param save: Si True, guarda los resultados en un archivo CSV
    :param save_path: Ruta donde se guardará el archivo CSV
    :param save_name: Nombre del archivo CSV
    :return: Array estructurado con campos 'sigma', 'mean' y 'std'
        - sigma: valores de ruido utilizados
        - mean: media de LCEs obtenida
        - std: desviación típica de LCEs
    """
    np.random.seed(2)

    # definimos los parametros del sistema
    r = np.array([1, 0.72, 1.53, 1.27])
    a = np.array([
        [1, 1.09, 1.52, 0],
        [0, 1, 0.44, 1.36],
        [2.33, 0, 1, 0.47],
        [1.21, 0.51, 0.35, 1]
    ])
    x0 = np.array([0.5, 0.5, 0.5, 0.5])

    # creamos algunas variables
    array_sigmas = np.linspace(start=0.0, stop=1.5, num=max_iters)

    # definimos dtype estructurado: sigma, mean, std
    dtype = [('sigma', float), ('mean', float), ('std', float)]
    resultados = np.zeros(max_iters, dtype=dtype)

    for idx, sigma in enumerate(array_sigmas):
        print(f"{idx+1}/{max_iters}  sigma={sigma:.3f}...", end='\r')

        # creamos el vector del ruido
        vect_ruido = sigma * np.array([0.8, 0.9, 1.0, 1.1])

        # resolvemos el sistema
        model = StochasticSystem(
            r, a, x0,
            total_time=total_time,
            dt=total_time / N_points,
            sigma=vect_ruido
        )
        T, X = model.euler_maruyama()

        # hallamos los LCEs con el método de Wolf
        lambda1, vect_lambdas = model.estimate_LCE1_Wolf(
            X, evol_time=500,
            min_separation=180,
            max_replacements=500,
            start_indices=N_points // 2
        )

        # hallamos la media y la std
        mean_val = np.nanmean(vect_lambdas)
        std_val = np.nanstd(vect_lambdas, ddof=1)

        # guardamos los resultados
        resultados['sigma'][idx] = sigma
        resultados['mean'][idx] = mean_val
        resultados['std'][idx] = std_val

    # guardado opcional
    if save and save_path and save_name:
        import pandas as pd
        df = pd.DataFrame(resultados)
        df.to_csv(f"{save_path}/{save_name}.csv", index=False)

    return resultados

In [ ]:
# prueba
barrido_test = barrido_LCEs(
    total_time=5000,
    N_points=1e5,
    max_iters=50
)